In [66]:
import sys
import os
from pathlib import Path
sys.path.append(str(Path.cwd().parent))

from core.constants import *
from core.strategy import Selector, SelectorType, SelectorSequence, Filter

target_key = TableKey.COMPANY_RATING

n_division = 4
lookback_year = 1

filter_list = []
relative_range_list = [(i / n_division, (i + 1) / n_division) for i in range(n_division)]
relative_range_list.append((0.0, 1.0))

for relative_value_range in relative_range_list:
    filter_name = target_key + '=(' + str(relative_value_range[0]) + '-' + str(relative_value_range[1]) + ')'
    relative_selector = Selector(type=SelectorType.GROWTH, key=target_key, value=relative_value_range,
                                 horizon=lookback_year,
                                 is_absolute=False) # vs GROWTH
    selector_sequence = SelectorSequence(selectors=[relative_selector])
    filter_list.append(Filter(selector_sequence_union=[selector_sequence], name=filter_name))

In [67]:
from datetime import date
from core.db_interface import DataBaseInterface
from core.strategy import Strategist
import pandas as pd
from tqdm import tqdm

db_interface = DataBaseInterface()

exchange_pool = ['NASDAQ','NYSE']
horizon = (date(2021, 3, 1), date(2022, 2, 25)) # 2021 년부터 테스트 해보기 !
freq = '1M'

initial_symbols = db_interface.get_stock_on_exchange(exchange_pool)
rebalance_dates = pd.date_range(horizon[0], horizon[1], freq=freq, inclusive='both')
strategist_list = []

tqdm_object = tqdm(total=len(filter_list) * len(rebalance_dates))

with tqdm_object as pbar:
    for filter_ in filter_list:
        strategist_name = filter_.name
        strategist = Strategist(fetcher=db_interface, name=strategist_name, asset_pool=initial_symbols,
                                start_date=horizon[0], end_date=horizon[1])
        for rebalance_date in rebalance_dates:
            strategist.apply_filter(filter_in=filter_, date_in=rebalance_date.date())
            pbar.update(1)
        strategist_list.append(strategist)


100%|██████████| 55/55 [00:00<00:00, 146.18it/s]


In [68]:
from core.evaluate import get_all_symbols_price_history

df_all_price_history = get_all_symbols_price_history(strategist_list[-1], db_interface)
df_all_price_history = df_all_price_history.resample('6D').mean()

179 / 179 종목이 유효한 가격 히스토리를 가지고 있습니다.


In [1]:
df_all_price_history.drop('JE',axis=1,inplace=True)

NameError: name 'df_all_price_history' is not defined

In [69]:
%matplotlib qt
import matplotlib.pyplot as plt
from core.evaluate import show_comparative_report, estimate_price_history, generate_sector_color, compute_analysis_series

price_history_dict = \
    {strategist.name: estimate_price_history(strategist, df_all_price_history)
     for strategist in strategist_list}


period = '1M'

fig, ax =plt.subplots(2,1,constrained_layout=True, figsize=(10, 10))
plt.rcParams['figure.dpi'] =100
plt.rcParams['font.size'] = 14
plt.rcParams['font.family'] = 'Malgun Gothic'
plt.rcParams['axes.unicode_minus'] = False
colors = generate_sector_color(list(price_history_dict.keys()), "husl")
series_list = []
for k, v in price_history_dict.items():
    ax1 = plt.subplot(2, 1, 1)
    plt.plot(v, label=k, color=colors[k])
    x_lim = plt.gca().get_xlim()
    plt.title('잔고의 변화 (초기 = 1원)')
    plt.legend()
    plt.grid(axis='y',color='0.8')
    ax1.set_facecolor('white')
    ax1.tick_params(color='white')
    for spine in ax1.spines.values():
        spine.set_edgecolor('white')

    ax2 = plt.subplot(2, 1, 2)
    pct_change = v.resample(period).last().pct_change() * 100
    plt.plot(pct_change, 'o-', color=colors[k], markersize=4, linewidth=0.8)
    plt.title(f'{period[:-1]}개월 수익률 (%)')
    plt.gca().set_xlim(x_lim)
    plt.grid(axis='both',color='0.8')
    ax2.set_facecolor('white')

    series_list.append(compute_analysis_series(v, k))
